[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MeteoSwiss/inca-examples/blob/main/notebooks/reprojecting_inca.ipynb)

# Reproject INCA data

In this jupyter notebook, we will show how to import and reproject a precipitation
forecast from the INCA [nowcasting system](https://www.meteoswiss.admin.ch/home/services-and-publications/produkte.subpage.html/en/data/products/2021/nowcasting-inca-ch.html).

## Install dependencies

First, let's set up our working environment. It is recommended to use a virtual
environment.

**Important**: In Google Colab, this notebook needs to be executed one cell
at a time. Trying to excecute all the cells at once may results in cells being 
skipped and some dependencies not being installed.

Now, let's install the  dependendies that will allow us to read and visualize the 
example data. We will need [xarray](http://xarray.pydata.org) to work with
labelled multi-dimensional arrays, [pyproj](https://github.com/pyproj4/pyproj)
for coordinate transformations, and [rioxarray](https://github.com/corteva/rioxarray)
to extend xarray with methods from [rasterio](https://rasterio.readthedocs.io) for geospatial datasets.


In [ ]:
!pip install xarray pyproj rioxarray

## Download the data

Now that we have the environment ready, let's download an example dataset.

On [Zenodo](https://zenodo.org/record/5575197), we can download the ouput from a
single run of INCA for precipitation (mm/h).

In [ ]:
!wget https://zenodo.org/record/5575197/files/RR_INCA.nc

## Read the data

Now that we have downloaded the example data, let's import the example netCDF 
with `xarray.open_dataset()` method.

The file is called `RR_INCA.nc`. "RR" is the product name used to identify
the quantitative precipitation nowcasting product based on the merging between
radar and station observations (named CombiPrecip). This product is in units of
mm/h, it is updated every 10 minutes, and it has a 6 hour forecast range.

We import this 6-hour precipitation forecast, which corresponds to a sequence of 37 
frames of 10-min precipitation fields (1 analysis and 36 forecasts).

In [ ]:
import xarray as xr

ds = xr.open_dataset("RR_INCA.nc")
ds

## Reproject with pyproj

The Swiss CH1903 / LV03 coordinates ([EPSG 21781](https://epsg.io/21781)) can be reprojected to any other reference system using pyproj. 
In the example below, we'll use WGS84 as the target projection ([EPSG 4326](https://epsg.io/4326)).

Note that as a result, the dataset won't be anymore on a regular grid.

In [ ]:
import numpy as np
from pyproj import Transformer

chxv, chyv = np.meshgrid(ds.chx, ds.chy)
transformer = Transformer.from_crs(
    "epsg:21781", 
    "epsg:4326",
    always_xy=True,
) 
lon, lat = transformer.transform(chxv, chyv)
ds = ds.assign_coords({"lon": (("chy", "chx"), lon) , "lat": (("chy", "chx"), lat)})
ds.RR.isel(time=0).plot.pcolormesh("lon", "lat")  # not a regular grid anymore

## Reproject and resample with rioxarray

The rioxarray library extends xarray with methods based on rasterio to handle georefenced dataset. This makes it very easy to reproject and resample datasets between different CRS.

In [ ]:
import rioxarray  # for the extension to load

ds = xr.open_dataset("RR_INCA.nc")
ds = ds.rio.write_crs("epsg:21781")
ds_latlon = ds.rio.reproject("epsg:4326", resolution=0.01)
ds_latlon.RR.isel(time=0).plot()  # still a regular grid
ds_latlon